In [75]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from PIL import ImageFile,Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
n
# Load MobileNetV2 for feature extraction
base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3564\3630600779.py:12: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


In [77]:
def extract_features(img_path):
    try:
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        x = np.expand_dims(x, axis=0)
        features = model.predict(x)
        return features[0]
    except Exception as e:
        print(f"❌ Failed to process {img_path}: {e}")
        return None

In [7]:
image_db = []  # Store (folder_name, image_path, feature_vector)

base_dir = 'laptop_phone_dataset'
for category in os.listdir(base_dir):  # resell/recycle/dispose
    category_path = os.path.join(base_dir, category)
    for ewaste_type in os.listdir(category_path):
        type_path = os.path.join(category_path, ewaste_type)
        for img_name in os.listdir(type_path):
            img_path = os.path.join(type_path, img_name)
            features = extract_features(img_path)
            if features is not None:
                image_db.append((category, ewaste_type, img_path, features))

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Admin\AppData\Local\Temp\ipykernel_3564\525703777.py:3: SyntaxWarning: invalid escape sequence '\d'
  base_dir = 'laptop_phone_dataset\dataset'


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 

In [35]:
import pickle

# Save image_db as a pickle file
with open("image_db.pkl", "wb") as f:
    pickle.dump(image_db, f)
print("done")

done


In [11]:
def classify_damage(input_img_path, detected_type):  # detected_type = laptop/mobile
    input_features = extract_features(input_img_path)
    best_match = None
    best_similarity = -1

    for category, ewaste_type, path, features in image_db:
        if ewaste_type != detected_type:
            continue  # Match only same type (e.g., laptop)

        similarity = cosine_similarity([input_features], [features])[0][0]
        if similarity > best_similarity: 
            best_similarity = similarity
            best_match = category  # resell/recycle/dispose

    return best_match

In [33]:
result = classify_damage('l6.jpg', 'Laptop')
print(f"Prediction → {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Prediction → dispose
